In [1]:
from dotenv import load_dotenv
import os 


load_dotenv()


OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

print(OPENAI_API_KEY)

sk-proj-yduIlTB2fj0SFevKALhHT3BlbkFJ6Q7Z2tUbDZkV6129JWOL


In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# https://smith.langchain.com/o/6459a739-5a87-57df-b2aa-7bf48e50a36b/projects?paginationState=%7B%22pageIndex%22%3A0%2C%22pageSize%22%3A10%7D
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("final_model")

LangSmith 추적을 시작합니다.
[프로젝트명]
final_model


In [3]:
# 예제 문서 
# PDF 문서 로드 및 텍스트 추출
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "../raw_data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf"

# loader = PyPDFLoader(file_path, extract_images=False)
loader = PyMuPDFLoader(file_path, extract_images=False)

docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 41


In [4]:
docs[0].page_content

'사례로 정리해보는\n한-인도 CEPA \n활용법 및 인도 통상 애로\nKOTRA자료 22-055\n'

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 58


### LLM모델 및 임베딩 모델준비 

In [6]:
# llm 모델 
from langchain_openai import ChatOpenAI
from config import config

llm = ChatOpenAI(
            model       = config['llm_predictor']['model_name'],            # gpt-3.5-turbo
            temperature = config['llm_predictor']['temperature']            # temperature" : 0
        )

# 임베딩 모델 
from langchain_openai import OpenAIEmbeddings

from config import config
embeddings = OpenAIEmbeddings( model=config['embed_model']['model_name'] )


### 벡터DB 및 리트리버 준비

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS

# vector_store = Chroma(
#             persist_directory=config["chroma"]["persist_dir"],      # "persist_dir" : "./database",
#             embedding_function=embeddings              # OpenAIEmbeddings( model=config['embed_model']['model_name'] )
#         )

# FAISS를 사용하여 문서와 임베딩으로부터 데이터베이스를 생성합니다.
database = FAISS.from_documents(docs, embeddings)



In [8]:
query = "원산지증명서 오류 발급 시 정정 방법을 알려줘"

In [9]:
test_doc = database.similarity_search(query)

# 결과를 출력합니다.
print(test_doc[0].page_content)

인도 수출 및 CEPA 활용 시 주의사항 알아두기 | 11
2. 원산지증명서
가. 원산지증명서 오류 발급 시 정정 방법
① 주요 내용
한-인도 CEPA 협정 4.4조에 의하면 수출국에서 선적일로부터 7일 내 원산지증명서를 발급하여야 하며, 7
일 이후에 발급된 원산지증명서의 경우에는 ‘소급적용(ISSUED RETROSPECTIVELY)’ 문구가 표시되어 있
어야 한다. 해당 내용이 협정과 다르게 표시될 경우, 원산지증명서 오류 발급으로 간주된다. 
② 사례
한국에서 인도로 스피커 부품을 수출하고 있는 B사는 기업 측의 착오로 한국 세관에 원산지증명서 발급 신
청 시 선적일로부터 5일이 되었음에도 ‘소급적용’ 문구 표시를 신청하여 발급받았다. 인도 세관에서는 한-인
도 CEPA 협정에 따라 해당 원산지증명서를 적합하지 않다고 판단하여, ‘소급적용’ 문구가 없는 원산지증명
서를 제출할 것을 요청하였다. 이와 같은 오류가 있는 경우, 한-인도 CEPA 적용을 위해 원산지증명서의 재
발급이 필요하나 재발급을 위해서는 기존에 발급된 원산지증명서 원본을 회수한 이후에 가능하므로 한-인도 
CEPA 적용이 늦어지게 되고, 통관 지연에 따른 보관료 등이 발생할 것으로 예상되었다. 뉴델리 CEPA활용
지원센터는 한국 세관과 관련 내용을 확인한 후, 다음과 같은 정정 발급을 통해 보다 빠르게 한-인도 CEPA
를 적용할 수 있도록 안내하였다.
1. 한국 세관에
원산지증명서 정정신청 
2. 정정 원산지증명서 제출,
한-인도 CEPA 적용
3. 원산지증명서
원본(오류 발급본) 회수
③ 시사점
자유무역협정의 이행을 위한 관세법의 특례에 관한 법률 제12조에 따라 관세당국에서 수출물품의 원산지
증명 능력이 있다고 인정받은 경우, 원산지증명서 발급이 간소화 된다. 해당 인증을 취득한 기업(원산지 인증 
수출자)은 원산지증명서 발급 신청 시 해당 물품의 원산지를 입증하기 위한 자료 제출이 생략되고, 발급 기관
의 심사도 간소화된다. 이렇게 인증 수출자 제도를 통해 원산지증명서 발급을 신청

In [10]:
for i in range(len(test_doc)):

    results = database.similarity_search_with_score(test_doc[i].page_content, k=1)

    print(f'유사도 검사 중...results : {results}')
    print(f'score: {results[0][1]}')

유사도 검사 중...results : [(Document(page_content='인도 수출 및 CEPA 활용 시 주의사항 알아두기 | 11\n2. 원산지증명서\n가. 원산지증명서 오류 발급 시 정정 방법\n① 주요 내용\n한-인도 CEPA 협정 4.4조에 의하면 수출국에서 선적일로부터 7일 내 원산지증명서를 발급하여야 하며, 7\n일 이후에 발급된 원산지증명서의 경우에는 ‘소급적용(ISSUED RETROSPECTIVELY)’ 문구가 표시되어 있\n어야 한다. 해당 내용이 협정과 다르게 표시될 경우, 원산지증명서 오류 발급으로 간주된다. \n② 사례\n한국에서 인도로 스피커 부품을 수출하고 있는 B사는 기업 측의 착오로 한국 세관에 원산지증명서 발급 신\n청 시 선적일로부터 5일이 되었음에도 ‘소급적용’ 문구 표시를 신청하여 발급받았다. 인도 세관에서는 한-인\n도 CEPA 협정에 따라 해당 원산지증명서를 적합하지 않다고 판단하여, ‘소급적용’ 문구가 없는 원산지증명\n서를 제출할 것을 요청하였다. 이와 같은 오류가 있는 경우, 한-인도 CEPA 적용을 위해 원산지증명서의 재\n발급이 필요하나 재발급을 위해서는 기존에 발급된 원산지증명서 원본을 회수한 이후에 가능하므로 한-인도 \nCEPA 적용이 늦어지게 되고, 통관 지연에 따른 보관료 등이 발생할 것으로 예상되었다. 뉴델리 CEPA활용\n지원센터는 한국 세관과 관련 내용을 확인한 후, 다음과 같은 정정 발급을 통해 보다 빠르게 한-인도 CEPA\n를 적용할 수 있도록 안내하였다.\n1. 한국 세관에\n원산지증명서 정정신청 \n2. 정정 원산지증명서 제출,\n한-인도 CEPA 적용\n3. 원산지증명서\n원본(오류 발급본) 회수\n③ 시사점\n자유무역협정의 이행을 위한 관세법의 특례에 관한 법률 제12조에 따라 관세당국에서 수출물품의 원산지\n증명 능력이 있다고 인정받은 경우, 원산지증명서 발급이 간소화 된다. 해당 인증을 취득한 기업(원산지 인증 \n수출자)은 원산지증명서 발급 신청 시 해당 물품의 원산지를 

In [11]:
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

# retriever = vector.as_retriever()
faiss_retriever = database.as_retriever(
            search_kwargs = {"k": config["retriever_k"]},           # 유사도 높은 순으로 가져올 벡터 개수 
            search_type   = config["search_type"]                        # 유사도 측정 방법 
        )

# BM25를 이용한 Sparce search(keyward 기반) retriever에 docs를 불러오기
bm25_retriever = BM25Retriever.from_documents(
    docs,
)
bm25_retriever.k = 5

# Ensenble retriever
ensemble_retriever = EnsembleRetriever(

    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.65, 0.35], # 가중치
    search_type=config['ensemble_search_type'], # mmr 기반
)

In [12]:
retrival_result1 = ensemble_retriever.get_relevant_documents(query)

retrival_result1

c:\Users\User\anaconda3\envs\llm\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='인도 수출 및 CEPA 활용 시 주의사항 알아두기 | 11\n2. 원산지증명서\n가. 원산지증명서 오류 발급 시 정정 방법\n① 주요 내용\n한-인도 CEPA 협정 4.4조에 의하면 수출국에서 선적일로부터 7일 내 원산지증명서를 발급하여야 하며, 7\n일 이후에 발급된 원산지증명서의 경우에는 ‘소급적용(ISSUED RETROSPECTIVELY)’ 문구가 표시되어 있\n어야 한다. 해당 내용이 협정과 다르게 표시될 경우, 원산지증명서 오류 발급으로 간주된다. \n② 사례\n한국에서 인도로 스피커 부품을 수출하고 있는 B사는 기업 측의 착오로 한국 세관에 원산지증명서 발급 신\n청 시 선적일로부터 5일이 되었음에도 ‘소급적용’ 문구 표시를 신청하여 발급받았다. 인도 세관에서는 한-인\n도 CEPA 협정에 따라 해당 원산지증명서를 적합하지 않다고 판단하여, ‘소급적용’ 문구가 없는 원산지증명\n서를 제출할 것을 요청하였다. 이와 같은 오류가 있는 경우, 한-인도 CEPA 적용을 위해 원산지증명서의 재\n발급이 필요하나 재발급을 위해서는 기존에 발급된 원산지증명서 원본을 회수한 이후에 가능하므로 한-인도 \nCEPA 적용이 늦어지게 되고, 통관 지연에 따른 보관료 등이 발생할 것으로 예상되었다. 뉴델리 CEPA활용\n지원센터는 한국 세관과 관련 내용을 확인한 후, 다음과 같은 정정 발급을 통해 보다 빠르게 한-인도 CEPA\n를 적용할 수 있도록 안내하였다.\n1. 한국 세관에\n원산지증명서 정정신청 \n2. 정정 원산지증명서 제출,\n한-인도 CEPA 적용\n3. 원산지증명서\n원본(오류 발급본) 회수\n③ 시사점\n자유무역협정의 이행을 위한 관세법의 특례에 관한 법률 제12조에 따라 관세당국에서 수출물품의 원산지\n증명 능력이 있다고 인정받은 경우, 원산지증명서 발급이 간소화 된다. 해당 인증을 취득한 기업(원산지 인증 \n수출자)은 원산지증명서 발급 신청 시 해당 물품의 원산지를 입증하기 위한 자료 제출이 생략되고, 발

In [13]:
retrival_result2 = ensemble_retriever.invoke(query)

retrival_result2

[Document(page_content='인도 수출 및 CEPA 활용 시 주의사항 알아두기 | 11\n2. 원산지증명서\n가. 원산지증명서 오류 발급 시 정정 방법\n① 주요 내용\n한-인도 CEPA 협정 4.4조에 의하면 수출국에서 선적일로부터 7일 내 원산지증명서를 발급하여야 하며, 7\n일 이후에 발급된 원산지증명서의 경우에는 ‘소급적용(ISSUED RETROSPECTIVELY)’ 문구가 표시되어 있\n어야 한다. 해당 내용이 협정과 다르게 표시될 경우, 원산지증명서 오류 발급으로 간주된다. \n② 사례\n한국에서 인도로 스피커 부품을 수출하고 있는 B사는 기업 측의 착오로 한국 세관에 원산지증명서 발급 신\n청 시 선적일로부터 5일이 되었음에도 ‘소급적용’ 문구 표시를 신청하여 발급받았다. 인도 세관에서는 한-인\n도 CEPA 협정에 따라 해당 원산지증명서를 적합하지 않다고 판단하여, ‘소급적용’ 문구가 없는 원산지증명\n서를 제출할 것을 요청하였다. 이와 같은 오류가 있는 경우, 한-인도 CEPA 적용을 위해 원산지증명서의 재\n발급이 필요하나 재발급을 위해서는 기존에 발급된 원산지증명서 원본을 회수한 이후에 가능하므로 한-인도 \nCEPA 적용이 늦어지게 되고, 통관 지연에 따른 보관료 등이 발생할 것으로 예상되었다. 뉴델리 CEPA활용\n지원센터는 한국 세관과 관련 내용을 확인한 후, 다음과 같은 정정 발급을 통해 보다 빠르게 한-인도 CEPA\n를 적용할 수 있도록 안내하였다.\n1. 한국 세관에\n원산지증명서 정정신청 \n2. 정정 원산지증명서 제출,\n한-인도 CEPA 적용\n3. 원산지증명서\n원본(오류 발급본) 회수\n③ 시사점\n자유무역협정의 이행을 위한 관세법의 특례에 관한 법률 제12조에 따라 관세당국에서 수출물품의 원산지\n증명 능력이 있다고 인정받은 경우, 원산지증명서 발급이 간소화 된다. 해당 인증을 취득한 기업(원산지 인증 \n수출자)은 원산지증명서 발급 신청 시 해당 물품의 원산지를 입증하기 위한 자료 제출이 생략되고, 발

In [15]:


for i in range(len(retrival_result1)):

    results = database.similarity_search_with_score(retrival_result2[i].page_content, k=1)

    print(f'유사도 검사 중...results : {results}')
    print(f'score: {results[0][1]}')

유사도 검사 중...results : [(Document(page_content='인도 수출 및 CEPA 활용 시 주의사항 알아두기 | 11\n2. 원산지증명서\n가. 원산지증명서 오류 발급 시 정정 방법\n① 주요 내용\n한-인도 CEPA 협정 4.4조에 의하면 수출국에서 선적일로부터 7일 내 원산지증명서를 발급하여야 하며, 7\n일 이후에 발급된 원산지증명서의 경우에는 ‘소급적용(ISSUED RETROSPECTIVELY)’ 문구가 표시되어 있\n어야 한다. 해당 내용이 협정과 다르게 표시될 경우, 원산지증명서 오류 발급으로 간주된다. \n② 사례\n한국에서 인도로 스피커 부품을 수출하고 있는 B사는 기업 측의 착오로 한국 세관에 원산지증명서 발급 신\n청 시 선적일로부터 5일이 되었음에도 ‘소급적용’ 문구 표시를 신청하여 발급받았다. 인도 세관에서는 한-인\n도 CEPA 협정에 따라 해당 원산지증명서를 적합하지 않다고 판단하여, ‘소급적용’ 문구가 없는 원산지증명\n서를 제출할 것을 요청하였다. 이와 같은 오류가 있는 경우, 한-인도 CEPA 적용을 위해 원산지증명서의 재\n발급이 필요하나 재발급을 위해서는 기존에 발급된 원산지증명서 원본을 회수한 이후에 가능하므로 한-인도 \nCEPA 적용이 늦어지게 되고, 통관 지연에 따른 보관료 등이 발생할 것으로 예상되었다. 뉴델리 CEPA활용\n지원센터는 한국 세관과 관련 내용을 확인한 후, 다음과 같은 정정 발급을 통해 보다 빠르게 한-인도 CEPA\n를 적용할 수 있도록 안내하였다.\n1. 한국 세관에\n원산지증명서 정정신청 \n2. 정정 원산지증명서 제출,\n한-인도 CEPA 적용\n3. 원산지증명서\n원본(오류 발급본) 회수\n③ 시사점\n자유무역협정의 이행을 위한 관세법의 특례에 관한 법률 제12조에 따라 관세당국에서 수출물품의 원산지\n증명 능력이 있다고 인정받은 경우, 원산지증명서 발급이 간소화 된다. 해당 인증을 취득한 기업(원산지 인증 \n수출자)은 원산지증명서 발급 신청 시 해당 물품의 원산지를 

### Chain

In [16]:
from prompt import contextualize_q_prompt, qa_prompt


from langchain.chains import create_history_aware_retriever
# 1. 사용자의 질문 문맥화 -> 대화 히스토리 체인 
history_aware_retriever = create_history_aware_retriever(
                                                        llm = llm,
                                                        retriever = ensemble_retriever,
                                                        prompt = contextualize_q_prompt          # 사용자 질문 맥락화 프롬프트: 채팅 기록의 맥락을 참조 -> 자연스러운 대화를 위해 
)

from langchain.chains.combine_documents import create_stuff_documents_chain
# 2. 응답 조건 프롬프트 체인 
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)                    # llm과 응답 조건 프롬프트 체인 

from langchain.chains import create_retrieval_chain
# 3. 최종 체인 생성 : 1과 2를 연결 
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [17]:
query = "인도 DRI(국세정보국) 관세조사 시 대응방안에 대한 사례를 알려주세요."
result = rag_chain.invoke(
    {
        "input": query,
        "chat_history": [],
    }
)

result['answer']

'인도 DRI(국세정보국) 관세조사 시 대응방안에 대한 사례는 다음과 같습니다:\n\n사례:\n한국 기업 A사는 인도에 수출하는 핸드폰 및 TV 등에 사용되는 스폰지 가스켓(Sponge Gasket) 부품을 생산하고 있습니다. A사는 한국에서 해당 부품의 HS 코드가 7410.21에 해당한다는 것을 확인하고, 약 1년 넘게 해당 HS 코드로 인도에 수출 및 한-인도 CEPA의 혜택을 받아왔습니다.\n\n그러나 어느 날, 인도 세관 담당자로부터 해당 부품의 HS 코드를 3919.90으로 변경하고 다시 수출 및 CEPA 적용을 요청하는 요구를 받았습니다. 이에 A사는 해당 HS 코드 변경으로 인한 CEPA 혜택 적용 여부를 확인하기 위해 대응해야 했습니다.\n\nA사는 다행히도 HS 코드 7410.21과 3919.90 모두 한-인도 CEPA 협정 상 양허대상 물품에 해당하고 세율도 0%로 동일했습니다. 따라서, 인도 세관의 요청에 따라 HS 코드를 변경하여도 CEPA 혜택을 계속 적용할 수 있었습니다.\n\n이 사례에서 A사는 인도 DRI(국세정보국)의 자의적인 품목분류 변경에 대응하기 위해 CEPA 혜택을 유지하면서도 적절히 대응하는 방법을 찾아냈습니다.\n\n이를 통해 기업은 인도 DRI의 관세조사에 대비하고 CEPA 혜택을 유지하는 방법을 찾아야 함을 알 수 있습니다.'

In [18]:
query = "원산지증명서 오류 발급 시 정정 방법을 알려줘"
result = rag_chain.invoke(
    {
        "input": query,
        "chat_history": [],
    }
)

result['answer']

"원산지증명서 오류 발급 시 정정 방법은 다음과 같습니다:\n\n1. 한-인도 CEPA 협정에 따르면 수출국에서 선적일로부터 7일 내에 원산지증명서를 발급하여야 합니다. 7일 이후에 발급된 원산지증명서의 경우에는 '소급적용(ISSUED RETROSPECTIVELY)' 문구가 표시되어 있어야 합니다. 해당 내용이 협정과 다르게 표시될 경우, 원산지증명서 오류 발급으로 간주됩니다.\n\n2. 원산지증명서 오류가 발생한 경우, 정정을 위해 다음 단계를 따를 수 있습니다:\n   - 한국 세관에 원산지증명서 정정신청을 합니다.\n   - 정정된 원산지증명서를 제출하고, 한-인도 CEPA 적용을 요청합니다.\n   - 원산지증명서의 원본(오류 발급본)을 회수합니다.\n\n3. 원산지증명서의 정정을 통해 보다 빠르게 한-인도 CEPA를 적용할 수 있도록 안내를 받을 수 있습니다.\n\n이러한 절차를 통해 원산지증명서 오류를 정정하여 한-인도 CEPA 협정을 적용할 수 있습니다."

In [19]:

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

session_histories = {} # 세션 기록을 저장할 딕셔너리

# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in session_histories:
        session_histories[session_id] = ChatMessageHistory()                            # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        print(f"[히스토리 생성] 새로운 히스토리를 생성합니다. 세션 ID: {session_id}")
    return session_histories[session_id]                                                 # 해당 세션 ID에 대한 세션 기록 반환


# LangChain Expression Language (LCEL)
# RunnableWithMessageHistory 클래스를 사용하여 chain과 메시지 기록을 연결합니다.

from langchain_core.runnables.history import RunnableWithMessageHistory

conversational_rag_chain = RunnableWithMessageHistory(
            rag_chain,                                  # 실행할 Runnable 객체 -> 체인LLM 모델 
            get_session_history,                        # 세션 기록을 가져오는 함수
            input_messages_key="input",                 # 입력 메시지의 키: chain을 invoke() 할때 사용자 쿼리 입력으로 지정하는 key
            history_messages_key="chat_history",        # # 기록 메시지의 키: 대화 기록으로 지정하는 key
            output_messages_key="answer"
        )

## 질문하기 

In [21]:
session_id = 'abc123'

question = "인도 DRI(국세정보국) 관세조사 시 대응방안에 대한 사례를 알려주세요."


In [22]:
response = conversational_rag_chain.invoke(
            {
                "input": question,
                "chat_history": [],
            },
            config={"configurable": {"session_id": session_id}}
        )


[히스토리 생성] 새로운 히스토리를 생성합니다. 세션 ID: abc123


In [23]:
response["answer"]

'**진출 사업:** 인도 DRI(국세정보국) 관세조사 시 대응방안\n\n**진출 사업 동향:** 인도 DRI(국세정보국)의 관세조사는 자동차 부품 관련 자의적 해석에 근거하여 시행되고 있습니다.\n\n**진출 사업 메리트:** 대비책을 마련하여 관세조사에 대응할 수 있다면, 한-인도 CEPA 활용 불가 및 차액 관세 납부 등의 피해를 최소화할 수 있습니다.\n\n**진출 사업 관련 정책:** 2022년 1월 5일, 인도 관세간접세위원회(CBIC)는 세관 및 국세정보국(DRI)의 자의적 품목분류를 규제하기 위한 지침을 발표하였습니다.\n\n**진출 사업을 위한 도움말:** 관련 기업들은 근거자료를 확보하고, HS 협약에 따른 규정을 우선 적용하며 다양한 사례를 종합적으로 참고하여 판단해야 합니다.\n\n**진출 사업 관련 알아야 할 것들:** 관세 관련 고시 및 지침은 인도 관세간접세위원회(CBIC)의 공식 웹사이트에서 확인할 수 있으며, 대비책을 마련하여 관세조사에 대응할 수 있어야 합니다.'